In [1]:
! lamin load jkobject/scprint2

→ connected lamindb: jkobject/scprint2


In [8]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)


from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes
import lamindb as ln

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Seed set to 42


→ connected lamindb: jkobject/scprint2


In [9]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_clss = [
    "cell_type_ontology_term_id",  # 1
    "tissue_ontology_term_id",
    "disease_ontology_term_id",  # 2
    "simplified_dev_stage",
    "assay_ontology_term_id",  # 3
    'self_reported_ethnicity_ontology_term_id',  # 4
]
clss_to_predict = hierarchical_clss+[
    'sex_ontology_term_id',  # 5
    "organism_ontology_term_id",  # 6
    "cell_culture"
]
clss_to_weight = [
    "tissue_ontology_term_id",
    "disease_ontology_term_id",
    "simplified_dev_stage",
    "assay_ontology_term_id",
    "organism_ontology_term_id",
    "clust_cell_type",
    # 'dataset_id',
    # 'cell_culture',
    #  "heat_diff",
    #  "total_counts",
    "nnz",
    #  "dpt_group",
]

gene_emb = '../data/main/gene_embeddings.parquet'
d_model = 128

In [10]:
datamodule = DataModule(
    collection_name="scPRINT-V2 test", #some, all, preprocessed dataset, all no zhang, 
    gene_embeddings=gene_emb,
    clss_to_weight=clss_to_weight,
    metacell_mode=True,
    clss_to_predict=clss_to_predict,
    hierarchical_clss=hierarchical_clss,
    organisms=["NCBITaxon:9606"],#, "NCBITaxon:10090"],
    how="random expr",
    max_len=3200,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category
    weight_scaler=100,
    batch_size=10,
    num_workers=12,
    train_oversampling_per_epoch=2,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    test_split=0.05)
testfiles = datamodule.setup()

! no run & transform got linked, call `ln.track()` & re-run
! run input wasn't tracked, call `ln.track()` and re-run
! run input wasn't tracked, call `ln.track()` and re-run
won't do any check but we recommend to have your dataset coming from local storage
100.0% are aligned
seeing a string: loading gene positions as biomart parquet file


In [11]:
model = scPrint(
    genes=datamodule.genes,
    d_model=d_model*2,
    nhead=2*2,
    num_heads_kv=2,
    nlayers=8,
    layers_cls = [d_model],
    classes = datamodule.classes,
    labels_hierarchy = datamodule.labels_hierarchy,
    dropout=0.1,
    transformer="flash",
    precpt_gene_emb=gene_emb,
    gene_pos_enc=datamodule.gene_pos,
    mvc_decoder="inner product",
    label_decoders = datamodule.decoders,
    fused_dropout_add_ln=False,
    num_batch_labels = datamodule.num_datasets,
    checkpointing=False,
    prenorm=True,
    #weight_decay=0.01,
    #zinb=False
)

In [ ]:
# from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test",
                           save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

# tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
# tlogger.log_graph(model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [12]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(
    do_denoise=True,
    noise=[0.6],
    do_cce=True,
    cce_sim=0.6,
    do_ecs=False,
    ecs_threshold=0.4,
    ecs_scale=0.05,
    class_scale=0.08,
    do_cls=True,
    do_mvc=False,
    do_adv_cls=False,
    run_full_forward=True,
    do_next_tp=False,
    mask_ratio=["TF"],
    warmup_duration=100,
    fused_adam=False,
    lr_reduce_patience=200,
)
trainer = Trainer(precision="16-mixed", gradient_clip_val=10, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1, reload_dataloaders_every_n_epochs=1000000, 
                  #logger=wandb_logger
                  )

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


In [11]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()


In [15]:
trainer.fit(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                            | Type                         | Params | Mode
----------------------------------------------------------------------------------------
0 | gene_encoder                    | GeneEncoder                  | 5.9 M  | eval
1 | expr_encoder                    | ContinuousValueEncoder       | 66.8 K | eval
2 | pos_encoder                     | PositionalEncoding           | 0      | eval
3 | class_encoder                   | CategoryValueEncoder         | 2.6 K  | eval
4 | depth_encoder                   | ContinuousValueEncoder       | 66.8 K | eval
5 | transformer                     | FlashTransformer             | 5.8 M  | eval
6 | expr_decoder                    | ExprDecoder                  | 133 K  | eval
7 | cls_decoders                    | ModuleDict                   | 315 K  | eval
8 | grad_reverse_discriminator_loss | AdversarialDiscriminatorLoss | 134 K  | eval
9 | mvc_decoder                     | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
no nan
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 2184, 2, 2, 64])
torch.Size([10, 2184, 4, 64]) torch.Size([10, 21

Training: |          | 0/? [00:00<?, ?it/s]

torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
torch.Size([10, 2041, 4, 64]) torch.Size([10, 2041, 2, 2, 64])
> /home/ml4ig1/Documents code/scPRINT/scprint/model/model.py(551)forward()
    549         else:
    550             print("no nan")
--> 551         if len(get_attention_layer) > 0:
    552             transformer_output, qkvs = transformer_output
    553             return (

tensor([     0,      0, 522496, 522496, 522496, 522496, 522496, 522496, 522496,
        522496], device='cuda:0')
tensor([69144., 14332., 40785., 91909., 93840.,  9891., 47531., 55657., 41583.,
         6959.], device=

In [14]:
trainingmode = TrainingMode(
    do_denoise=True,
    noise=[0.6],
    do_cce=True,
    cce_sim=0.6,
    do_ecs=False,
    ecs_threshold=0.4,
    ecs_scale=0.05,
    class_scale=0.08,
    do_cls=True,
    do_mvc=False,
    do_adv_cls=False,
    run_full_forward=True,
    do_next_tp=False,
    mask_ratio=["TF"],
    fused_adam=False,
    lr_reduce_monitor=None,
    
)
overfit_trainer = Trainer(precision="16-mixed", gradient_clip_val=10, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=10_000, overfit_batches=1, 
                  reload_dataloaders_every_n_epochs=1_000_000, num_sanity_val_steps=2)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [15]:
overfit_trainer.fit(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                            | Type                         | Params | Mode
----------------------------------------------------------------------------------------
0 | gene_encoder                    | GeneEncoder                  | 5.9 M  | eval
1 | expr_encoder                    | ContinuousValueEncoder       | 66.8 K | eval
2 | pos_encoder                     | PositionalEncoding           | 0      | eval
3 | class_encoder                   | CategoryValueEncoder         | 2.6 K  | eval
4 | depth_encoder                   | ContinuousValueEncoder       | 66.8 K | eval
5 | transformer                     | FlashTransformer             | 6.3 M  | eval
6 | expr_decoder                    | ExprDecoder                  | 133 K  | eval
7 | cls_decoders                    | ModuleDict                   | 315 K  | eval
8 | grad_reverse_discriminator_loss | AdversarialDiscriminatorLoss | 134 K  | eval
9 | mvc_decoder                     | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7f05124dc0>
Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/ml4ig1/miniconda3/envs/test/lib/python3.10/selectors.py", line 416, in select
    fd_